In [117]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

import requests  # API를 이용해 자료를 받아오기 위해
import json
import time
import math


# 데이터 분석 라이브러리
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
# !pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer

# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import folium
from folium.plugins import MarkerCluster, MiniMap

# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'

from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import json

# MySQL 연결 설정
host = 'da-4-1-db.c1widrnxppoz.ap-northeast-2.rds.amazonaws.com'  # 여기에 MySQL 서버의 호스트 주소 또는 IP 주소를 입력하세요
port = 3306
user = 'root'   
password = '9juvtDV9gG&[!'
database = 'JEJU_DB'
conn = pymysql.connect(host=host, port=port, user=user, password=password, database=database)

def Search_table_info():
    # 커서 생성
    cursor = conn.cursor()
    # 사용할 스키마 선택
    cursor.execute(f"USE {database};")

    # 선택된 스키마에 있는 테이블 목록 조회
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()

    # 선택된 스키마에 있는 테이블 목록 출력
    print(f"Tables in schema '{database}':")
    for table in tables:
        print(table[0])

    # 연결 종료
    # conn.close()

def to_dataFrame(table_name):
    # 사용할 스키마 선택
    cursor = conn.cursor()
    cursor.execute(f"USE {database};")

    # 테이블 데이터 가져오기
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql(query, conn)

    return df

def close_sql_server():
    conn.close()

## 제주

In [128]:
tracks=["외도 - 백록초 - 제주도청 - 제주시청 - 화북 - 삼양 - 동문로터리 - 제주공항 - 외도 "]

In [129]:
print(tracks)

['외도 - 백록초 - 제주도청 - 제주시청 - 화북 - 삼양 - 동문로터리 - 제주공항 - 외도 ']


In [130]:
def track_to_list(track):
    return [i.strip()+", 제주시" for i in track.split("-")]

In [131]:
address_lists = [track_to_list(i) for i in tracks]
# 510, 531, 615, 520, 625 순
len(address_lists)

1

In [132]:
import folium

from geopy.geocoders import Nominatim

# 각 선에 대한 색상
# colors = ['red', 'orange', 'green', 'blue', 'purple']
colors = ['red', 'orange', 'green', 'blue', 'purple', 'pink', 'brown', 'gray']


# geolocator 객체 생성
geolocator = Nominatim(user_agent="my_geocoder")

# 맵 생성
map_jeju = folium.Map(location=[33.2529607630432, 126.56101770909784], zoom_start=11)

# 주소 리스트 별로 선 그리기
for idx, address_list in enumerate(address_lists):
    route_coordinates = []
    for place in address_list:
        location = geolocator.geocode(place)
        if location:
            route_coordinates.append([location.latitude, location.longitude])
    if len(route_coordinates) > 1:
        folium.PolyLine(locations=route_coordinates, color=colors[idx], weight=2.5, opacity=1).add_to(map_jeju)

# 지도 출력
map_jeju

In [133]:
# 5개의 리스트를 하나의 리스트로 합치기
combined_list = []
for sublist in address_lists:
    combined_list.extend(sublist)

# 중복되는 값을 찾기
duplicates = set([x for x in combined_list if combined_list.count(x) > 1])

print("중복되는 값들:", duplicates)

중복되는 값들: {'외도, 제주시'}


In [134]:
# 중복되는 값의 개수를 세는 딕셔너리 생성
count_dict = {}
for item in combined_list:
    if item in count_dict:
        count_dict[item] += 1
    else:
        count_dict[item] = 1

# 3개 이상 중복되는 값을 찾기
three_or_more_duplicates = [key for key, value in count_dict.items() if value >= 3]

print("3개 이상 중복되는 값들:", three_or_more_duplicates)


3개 이상 중복되는 값들: []


In [135]:
# geolocator 객체 생성
geolocator = Nominatim(user_agent="my_geocoder")

# 맵 생성
map_jeju = folium.Map(location=[33.2529607630432, 126.56101770909784], zoom_start=11)

# 중복된 값들에 대해 선과 마커 표시하기
for location in three_or_more_duplicates:
    geocode = geolocator.geocode(location)
    if geocode:
        folium.Marker([geocode.latitude, geocode.longitude], popup=location, icon=folium.Icon(color='orange')).add_to(map_jeju)

# 주소 리스트 별로 선 그리기
for idx, address_list in enumerate(address_lists):
    route_coordinates = []
    for place in address_list:
        location = geolocator.geocode(place)
        if location:
            route_coordinates.append([location.latitude, location.longitude])
    if len(route_coordinates) > 1:
        folium.PolyLine(locations=route_coordinates, color=colors[idx], weight=2.5, opacity=1).add_to(map_jeju)

# 지도 출력
map_jeju


In [140]:
# geolocator 객체 생성
geolocator = Nominatim(user_agent="my_geocoder")

# 맵 생성
map_jeju = folium.Map(location=[33.2529607630432, 126.56101770909784], zoom_start=11)

# 모든 값에 대해 선과 마커 표시하기
for address_list in address_lists:
    route_coordinates = []
    for place in address_list:
        location = geolocator.geocode(place)
        if location:
            route_coordinates.append([location.latitude, location.longitude])
            folium.Marker([location.latitude, location.longitude], popup=place).add_to(map_jeju)  # 마커 추가
    if len(route_coordinates) > 1:
        folium.PolyLine(locations=route_coordinates, color='blue', weight=2.5, opacity=1).add_to(map_jeju)

# 지도 출력
map_jeju


## 서귀포

In [ ]:
tracks = [
    "컨벤션센터-중문-하원-용흥동-서귀포터미널-서귀포여고-중앙로터리-비석거리-효돈-공천포-위미-남원(남원→남주고오전운행)",
    "화순-안덕계곡-감산-창천-예래-중문-하원-용흥-서귀포터미널-서귀포여고-중앙로터리-서귀포고-서귀포향토오일시장-법호촌-하례입구",
    "하례입구-신례초-공천포-효돈중-토평초-오일시장-동문로터리-서귀포고-주공6단지-오일시장-토평-공천포-하례입구",
    "하례입구-서귀포농업기술센터-동상효-비석거리-중앙로터리-천지동주민센터-동문로터리-삼성여고-효돈중-공천포-대화동-(위미1리-위미중-상위미)-휴애리공원-하례입구",
    "천지연-동문로터리-중앙로터리-주공6단지-고근산-서귀포중앙도서관-월드컵경기장-법환초-수모루-서귀포여고-서문로터리-천지연",
    "회수-중문우체국-중문고-강정초-법환초-수모루-구터미널-동문로터리-서복전시관",
    "평안전문요양원-오일시장-중앙로터리-서귀여고-법환초-하원초-중문고-중문우체국-중문관광단지-큰솔동산-대포포구"
]

In [ ]:
print(tracks)

['컨벤션센터-중문-하원-용흥동-서귀포터미널-서귀포여고-중앙로터리-비석거리-효돈-공천포-위미-남원(남원→남주고오전운행)', '화순-안덕계곡-감산-창천-예래-중문-하원-용흥-서귀포터미널-서귀포여고-중앙로터리-서귀포고-서귀포향토오일시장-법호촌-하례입구', '하례입구-신례초-공천포-효돈중-토평초-오일시장-동문로터리-서귀포고-주공6단지-오일시장-토평-공천포-하례입구', '하례입구-서귀포농업기술센터-동상효-비석거리-중앙로터리-천지동주민센터-동문로터리-삼성여고-효돈중-공천포-대화동-(위미1리-위미중-상위미)-휴애리공원-하례입구', '천지연-동문로터리-중앙로터리-주공6단지-고근산-서귀포중앙도서관-월드컵경기장-법환초-수모루-서귀포여고-서문로터리-천지연', '회수-중문우체국-중문고-강정초-법환초-수모루-구터미널-동문로터리-서복전시관', '평안전문요양원-오일시장-중앙로터리-서귀여고-법환초-하원초-중문고-중문우체국-중문관광단지-큰솔동산-대포포구']


In [ ]:
def track_to_list(track):
    return [i.strip()+", 서귀포시" for i in track.split("-")]

In [ ]:
address_lists = [track_to_list(i) for i in tracks]
# 510, 531, 615, 520, 625 순

In [ ]:
import folium

from geopy.geocoders import Nominatim

# 각 선에 대한 색상
colors = ['red', 'orange', 'green', 'blue', 'purple','pink', 'brown']

# geolocator 객체 생성
geolocator = Nominatim(user_agent="my_geocoder")

# 맵 생성
map_jeju = folium.Map(location=[33.2529607630432, 126.56101770909784], zoom_start=11)

# 주소 리스트 별로 선 그리기
for idx, address_list in enumerate(address_lists):
    route_coordinates = []
    for place in address_list:
        location = geolocator.geocode(place)
        if location:
            route_coordinates.append([location.latitude, location.longitude])
    if len(route_coordinates) > 1:
        folium.PolyLine(locations=route_coordinates, color=colors[idx], weight=2.5, opacity=1).add_to(map_jeju)

# 지도 출력
map_jeju

In [ ]:
# 5개의 리스트를 하나의 리스트로 합치기
combined_list = []
for sublist in address_lists:
    combined_list.extend(sublist)

# 중복되는 값을 찾기
duplicates = set([x for x in combined_list if combined_list.count(x) > 1])

print("중복되는 값들:", duplicates)

중복되는 값들: {'중문고, 서귀포시', '동문로터리, 서귀포시', '비석거리, 서귀포시', '천지연, 서귀포시', '중문우체국, 서귀포시', '서귀포고, 서귀포시', '서귀포터미널, 서귀포시', '중앙로터리, 서귀포시', '효돈중, 서귀포시', '주공6단지, 서귀포시', '하원, 서귀포시', '중문, 서귀포시', '하례입구, 서귀포시', '수모루, 서귀포시', '법환초, 서귀포시', '공천포, 서귀포시', '서귀포여고, 서귀포시', '오일시장, 서귀포시'}


In [ ]:
# 중복되는 값의 개수를 세는 딕셔너리 생성
count_dict = {}
for item in combined_list:
    if item in count_dict:
        count_dict[item] += 1
    else:
        count_dict[item] = 1

# 3개 이상 중복되는 값을 찾기
three_or_more_duplicates = [key for key, value in count_dict.items() if value >= 3]

print("3개 이상 중복되는 값들:", three_or_more_duplicates)


3개 이상 중복되는 값들: ['서귀포여고, 서귀포시', '중앙로터리, 서귀포시', '공천포, 서귀포시', '하례입구, 서귀포시', '오일시장, 서귀포시', '동문로터리, 서귀포시', '법환초, 서귀포시']


In [ ]:
# geolocator 객체 생성
geolocator = Nominatim(user_agent="my_geocoder")

# 맵 생성
map_jeju = folium.Map(location=[33.2529607630432, 126.56101770909784], zoom_start=11)

# 중복된 값들에 대해 선과 마커 표시하기
for location in three_or_more_duplicates:
    geocode = geolocator.geocode(location)
    if geocode:
        folium.Marker([geocode.latitude, geocode.longitude], popup=location, icon=folium.Icon(color='orange')).add_to(map_jeju)

# 주소 리스트 별로 선 그리기
for idx, address_list in enumerate(address_lists):
    route_coordinates = []
    for place in address_list:
        location = geolocator.geocode(place)
        if location:
            route_coordinates.append([location.latitude, location.longitude])
    if len(route_coordinates) > 1:
        folium.PolyLine(locations=route_coordinates, color=colors[idx], weight=2.5, opacity=1).add_to(map_jeju)

# 지도 출력
map_jeju


In [ ]:
three_or_more_duplicates[5],three_or_more_duplicates[2]=three_or_more_duplicates[2],three_or_more_duplicates[5]

In [ ]:
# geolocator 객체 생성
geolocator = Nominatim(user_agent="my_geocoder")

# 맵 생성
map_duplicated = folium.Map(location=[33.2529607630432, 126.56101770909784], zoom_start=11)
count=0
# 중복된 값들에 대해 선과 마커 추가
prev_location = None
for location in three_or_more_duplicates:
    geocode = geolocator.geocode(location)
    if geocode:
        current_location = [geocode.latitude, geocode.longitude]
        # 마커 추가
        folium.Marker(current_location, popup="<pre>"+location+" </pre>").add_to(map_duplicated)
        if prev_location is not None:
            # 선 추가
            folium.PolyLine(locations=[prev_location, current_location], color="orange", weight=2.5, opacity=1).add_to(map_duplicated)
        prev_location = current_location
        print(count,location)
        count+=1
    else:
        print(location)

# 지도 출력
map_duplicated



서귀포여고, 서귀포시
0 중앙로터리, 서귀포시
1 동문로터리, 서귀포시
2 하례입구, 서귀포시
오일시장, 서귀포시
3 공천포, 서귀포시
법환초, 서귀포시
